### Potential interesting Questions:
- What does influence the rating?
    - Does game length ~?
    - Do prizes (honors column) ~?

rows dropped
- accessory rank, because I'm not interested in accessories

columns dropped containing  
- version
- compilation
- accessory


[Kaggle link](https://www.kaggle.com/datasets/seanthemalloy/board-game-geek-database/)

[API description](https://boardgamegeek.com/wiki/page/BGG_XML_API2#toc3)

In [ ]:
import numpy as np
import pandas as pd
import os
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
data_folder = "data"
file = os.path.join(data_folder, "BGGdata.csv")

In [ ]:
if os.path.isfile(file):
    bgg_df = pd.read_csv(file);
else:
    os.system("unzip " + file + ".zip -d " + data_folder)
    bgg_df = pd.read_csv(file);

In [ ]:
bgg_df2 = bgg_df.copy()

In [ ]:
def drop_columns(df:pd.DataFrame, search_string:list = None)-> None:
    if search_string:
        search_string =  search_string if type(search_string) == str else "(?i)"+"|".join(search_string)
        df_temp = df.filter(regex=search_string)
        drop_column = df_temp.columns
        df.drop(columns=drop_column, inplace = True)
        return None

    search_string = input("filter string:")
    df_temp = df.filter(like=search_string)
    drop_column = df_temp.columns #if regex else df.filter(like=search_string).columns
    display(drop_column)
    
    answer = input("drop? yes/display/exit/")

    if answer == "yes":
        df.drop(columns=drop_column, inplace = True)
        drop_columns(df)
    elif answer == "display":
        display(df[drop_column].dropna().head())
        drop_columns(df)
    elif answer == "exit":
        return None
    else:
        drop_columns(df)

In [ ]:
# Honor count is more interesting than which honor
bgg_df2["hon_count"] = bgg_df.filter(like="honor").count(axis=1)

In [ ]:
bgg_df2.filter(like="0").columns

In [ ]:
#bgg_df["categories"] = 
categories = {y for y in bgg_df.filter(like="category").values.flatten()}

In [ ]:
mechanics = {y for y in bgg_df.filter(like="mechanic").values.flatten()}


In [ ]:
families = {y for y in bgg_df.filter(like="family").values.flatten()}

In [ ]:
subdomains =  {y for y in bgg_df.filter(like="subdomain").values.flatten()}

In [ ]:
len(mechanics)

In [ ]:
# dropping uninteresting columns
drop_columns(bgg_df2, ["version","compilation","accessory", "implementation", "expansion", "thumbnail","commerce","honor"])

In [ ]:
drop_columns(bgg_df2)

In [ ]:
display(bgg_df2.filter(regex="0").dropna(how="all").columns)
bgg_df2.value_counts(subset="publisher0").head(27)